### Simulating a Quantum Computer with Python

Date: 24-01-2024

#### The structure

A few questions to get started:
- Where is the information of the system stored?
- How do we interact with the quantum computer? i.e. Change the system

#### The states

In [152]:
import numpy as np

# for a one qubit system, the state is a vector of length 2 We could initialise the system to be in the state |0> by setting the state vector to be [1,0].
state = np.array([1,0], dtype=np.complex128)


#### The gates

In [153]:
I = np.array(np.eye(2))
H = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
S = np.array([[1, 0], [0, 1j]])

operators = {'I':I, 'H':H, 'X':X, 'Y':Y, 'Z':Z, 'S':S}


Check if they are Hermitian and unitary!

In [154]:
def is_hermitian(operator, rtol=1e-05, atol=1e-08):
    op_mat = np.matrix(operator)
    ''' atol: absolute tolerance; 
        rtol: relative tolerance.'''
    return np.allclose(op_mat, op_mat.getH(), rtol=rtol, atol=atol)

def is_unitary(operator, rtol=1e-05, atol=1e-08):
    op_mat = np.matrix(operator)
    ''' atol: absolute tolerance; 
        rtol: relative tolerance.'''
    return np.allclose(op_mat @ op_mat.getH(), I, rtol=rtol, atol=atol)



In [155]:
for operator in operators.keys():
    print(operator)
    if is_hermitian(operators[operator]):
        print('is hermitian')
    if is_unitary(operators[operator]):
        print('is unitary')


I
is hermitian
is unitary
H
is hermitian
is unitary
X
is hermitian
is unitary
Y
is hermitian
is unitary
Z
is hermitian
is unitary
S
is unitary


Notice how the Phase gate S is not Hermitian? Why?

#### The Operations

In [156]:
state = np.array([1,0], dtype=np.complex128)
H_state = operators['H'] @ state
S_state = operators['S'] @ state

#### Some simple visualisation

In [157]:
def to_braket(state):
    # Find the indices of the states with non-zero amplitudes
    
    n_qubits = int(np.sqrt(len(state))) # calculate the number of qubits
    non_zero_indices = np.where(np.abs(state) > 0)[0]
    # Convert each non-zero index to binary representation, along with the corresponding coefficient
    computational_str = ""
    np.zeros(2**n_qubits, dtype=np.complex_)
    for idx in non_zero_indices:
        binary_str = format(idx, f"0{n_qubits}b")
        amplitude = state[idx]
        computational_str += f"{amplitude:.2f}|{binary_str}⟩ + "

    computational_str = computational_str.rstrip("+ ")

    return computational_str


In [158]:
to_braket(state)
to_braket(H_state)
to_braket(S_state)

'1.00+0.00j|0⟩'

#### The measurement


In [162]:
from collections import Counter

def measure(state, n_shots=1):

    probability = np.abs(state**2)
    n_qubits = int(np.sqrt(len(state))) 
    allowed_outcomes = np.arange(len(state))
        # print(state)
    outcomes = np.random.choice(allowed_outcomes, p=probability, size = n_shots)
    
    state = np.zeros_like(state)
    state[outcomes[-1]] = 1
    counts = Counter(outcomes)
    print(counts)
    outcomes_count = np.zeros((len(state),2),dtype=object) # 2: state and count
    for i in range(len(state)):
        if i in counts:
            outcomes_count[i] = counts[i], format(i, f"0{n_qubits}b")
        else:
            outcomes_count[i] = 0,format(i, f"0{n_qubits}b")
                
    return outcomes_count

measure(state, n_shots=1000)
measure(H_state, n_shots=1000)

Counter({0: 1000})
Counter({0: 518, 1: 482})


array([[518, '0'],
       [482, '1']], dtype=object)

When writing your own code for two or more qubits, consider:
- How does the statevector change as the system grows?
- How do the operators change?

Hint: $\otimes$


### The Simple Hamiltonian

Show that the Pauli matrices and the identity matrix form a basis for the $M_{2\times 2}(\mathbb{C})$ matrices, mathematically.

$$ M = 
\begin{bmatrix}
    a & b \\
    c & d
\end{bmatrix}
$$
and
$$ M = \alpha I + \beta X + \gamma Y + \delta Z $$